In [1]:
import mlflow
mlflow.set_tracking_uri("http://ec2-56-228-4-230.eu-north-1.compute.amazonaws.com:5000/")

In [2]:
# Set or create an experiment
mlflow.set_experiment("Exp 4 - Handling Imbalanced Data")

2025/11/23 22:22:56 INFO mlflow.tracking.fluent: Experiment with name 'Exp 4 - Handling Imbalanced Data' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://artifacts-sentiment-analysis/467995665707054504', creation_time=1763916776482, experiment_id='467995665707054504', last_update_time=1763916776482, lifecycle_stage='active', name='Exp 4 - Handling Imbalanced Data', tags={}>

In [4]:
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [5]:
df = pd.read_csv('../data/processed/cleaned_comments.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [6]:
# Step 1: Function to run the experiment
def run_imbalanced_experiment(imbalance_method):
    ngram_range = (1, 3)  # Trigram setting
    max_features = 10000  # Set max_features to 1000 for TF-IDF

    # Step 4: Train-test split before vectorization and resampling
    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

    # Step 2: Vectorization using TF-IDF, fit on training data only
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
    X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
    X_test_vec = vectorizer.transform(X_test)  # Transform test data

    # Step 3: Handle class imbalance based on the selected method (only applied to the training set)
    if imbalance_method == 'class_weights':
        # Use class_weight in Random Forest
        class_weight = 'balanced'
    else:
        class_weight = None  # Do not apply class_weight if using resampling

        # Resampling Techniques (only apply to the training set)
        if imbalance_method == 'oversampling':
            smote = SMOTE(random_state=42)
            X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)
        elif imbalance_method == 'adasyn':
            adasyn = ADASYN(random_state=42)
            X_train_vec, y_train = adasyn.fit_resample(X_train_vec, y_train)
        elif imbalance_method == 'undersampling':
            rus = RandomUnderSampler(random_state=42)
            X_train_vec, y_train = rus.fit_resample(X_train_vec, y_train)
        elif imbalance_method == 'smote_enn':
            smote_enn = SMOTEENN(random_state=42)
            X_train_vec, y_train = smote_enn.fit_resample(X_train_vec, y_train)

    # Step 5: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"Imbalance_{imbalance_method}_RandomForest_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "imbalance_handling")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with TF-IDF Trigrams, imbalance handling method={imbalance_method}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("imbalance_method", imbalance_method)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42, class_weight=class_weight)
        model.fit(X_train_vec, y_train)

        # Step 6: Make predictions and log metrics
        y_pred = model.predict(X_test_vec)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, Imbalance={imbalance_method}")
        confusion_matrix_filename = f"confusion_matrix_{imbalance_method}.png"
        plt.savefig(confusion_matrix_filename)
        mlflow.log_artifact(confusion_matrix_filename)
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_trigrams_imbalance_{imbalance_method}")


In [7]:
# Step 7: Run experiments for different imbalance methods
imbalance_methods = ['class_weights', 'oversampling', 'adasyn', 'undersampling', 'smote_enn']

for method in imbalance_methods:
    run_imbalanced_experiment(method)

2025/11/23 22:45:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/23 22:45:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_class_weights_RandomForest_TFIDF_Trigrams at: http://ec2-56-228-4-230.eu-north-1.compute.amazonaws.com:5000/#/experiments/467995665707054504/runs/2e9d76c0276a4dea85ee83c8216e7e94
🧪 View experiment at: http://ec2-56-228-4-230.eu-north-1.compute.amazonaws.com:5000/#/experiments/467995665707054504


2025/11/23 22:46:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/23 22:46:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_oversampling_RandomForest_TFIDF_Trigrams at: http://ec2-56-228-4-230.eu-north-1.compute.amazonaws.com:5000/#/experiments/467995665707054504/runs/ff6710ffa09647d1bafdfb92d07dd900
🧪 View experiment at: http://ec2-56-228-4-230.eu-north-1.compute.amazonaws.com:5000/#/experiments/467995665707054504


2025/11/23 22:47:50 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/23 22:48:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_adasyn_RandomForest_TFIDF_Trigrams at: http://ec2-56-228-4-230.eu-north-1.compute.amazonaws.com:5000/#/experiments/467995665707054504/runs/862275d85038421b98ecb21e28317220
🧪 View experiment at: http://ec2-56-228-4-230.eu-north-1.compute.amazonaws.com:5000/#/experiments/467995665707054504


2025/11/23 22:48:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/23 22:48:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_undersampling_RandomForest_TFIDF_Trigrams at: http://ec2-56-228-4-230.eu-north-1.compute.amazonaws.com:5000/#/experiments/467995665707054504/runs/3197aeb0d83646ab8a46caacb558ff20
🧪 View experiment at: http://ec2-56-228-4-230.eu-north-1.compute.amazonaws.com:5000/#/experiments/467995665707054504


2025/11/23 22:50:19 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/23 22:50:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_smote_enn_RandomForest_TFIDF_Trigrams at: http://ec2-56-228-4-230.eu-north-1.compute.amazonaws.com:5000/#/experiments/467995665707054504/runs/b46db7bce0084ab985698efd1f393723
🧪 View experiment at: http://ec2-56-228-4-230.eu-north-1.compute.amazonaws.com:5000/#/experiments/467995665707054504
